In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from tqdm import tqdm
import numpy as np
import networkx as nx

In [2]:
import sys
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers")
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\")
sys.path

['d:\\projects\\affect\\TUCORE-GCN\\experiments',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv',
 '',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\win32',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\win32\\lib',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\Pythonwin',
 'd:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers',
 'd:\\projects\\affect\\TUCORE-GCN\\']

In [3]:
from data import TUCOREGCNDataset, TUCOREGCNDataloader
from tucore_gcn_transformers.tucore_gcn_bert_train import TUCOREGCNDialogREDataset
from models.BERT.tokenization import FullTokenizer
tokenizer = FullTokenizer(vocab_file="../pre-trained_model/BERT/vocab.txt", do_lower_case=True)
old_dataset = TUCOREGCNDataset("../datasets/DialogRE/debug", "../datasets/DialogRE/debug/dev_BERT.pkl", 512, tokenizer, 36, "BERT")
old_loader = TUCOREGCNDataloader(dataset=old_dataset, batch_size=12, shuffle=False, relation_num=36, max_length=512)
old_data = old_loader.dataset.data
new_dataset = TUCOREGCNDialogREDataset()
new_data_generator = new_dataset._generate_examples('../datasets/DialogRE/debug/dev.json', 'dev', 512, False, True, False)
new_data = [data[1] for data in new_data_generator]

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\dgl\dgl.dll


01/11/2024 00:54:24 - INFO - datasets -   PyTorch version 2.0.1+cu118 available.


Reading data from ../datasets/DialogRE/debug.
load preprocessed data from ../datasets/DialogRE/debug/dev_BERT.pkl.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


  0%|          | 0/1 [00:00<?, ?it/s]

['speaker', '2', ':', "'", 'annie', '!', 'hi', '.', 'listen', 'we', 'got', 'a', 'problem', 'with', 'joey', 'tri', '##bb', '##iani', ',', 'apparently', 'he', 'missed', 'his', 'audition', '.', 'who', 'did', 'you', 'speak', 'to', 'in', 'my', 'office', '?', 'este', '##lle', ',', 'no', ',', 'i', 'don', "'", 't', 'know', 'what', 'i', "'", 'm', 'going', 'to', 'do', 'with', 'her', '.', 'no', '.', 'all', 'right', ',', 'so', 'your', 'husband', 'leaves', 'and', 'burns', 'down', 'the', 'apartment', ',', 'the', 'world', 'does', 'not', 'stop', '.', "'"]
437
75
76
['speaker', '2', ':', "'", 'annie', '!', 'hi', '.', 'listen', 'we', 'got', 'a', 'problem', 'with', 'joey', 'tri', '##bb', '##iani', ',', 'apparently', 'he', 'missed', 'his', 'audition', '.', 'who', 'did', 'you', 'speak', 'to', 'in', 'my', 'office', '?', 'este', '##lle', ',', 'no', ',', 'i', 'don', "'", 't', 'know', 'what', 'i', "'", 'm', 'going', 'to', 'do', 'with', 'her', '.', 'no', '.', 'all', 'right', ',', 'so', 'your', 'husband', 'leave

In [4]:
list(new_data[1].keys())[2:-2]

['input_ids', 'input_mask', 'segment_ids', 'speaker_ids', 'mention_ids']

In [5]:
from difflib import ndiff

for idx in range(len(old_data)):
	out = []
	cmp_all = list(new_data[idx].keys())[2:-2] #tokens, label_ids, turn_masks, graphs excluded
	for cmp in cmp_all:
		out.append(f"------------|{cmp}|------------")
		a = "".join([chr(code) for code in (old_data[idx][cmp if not (cmp=="mention_ids" or cmp=="turn_masks") else ("mention_id" if cmp=="mention_ids" else "turn_mask")])])
		b = "".join([chr(code) for code in new_data[idx][cmp]])
		for i,s in enumerate(ndiff(b,a)):
			if s[0]==' ': continue
			elif s[0]=='-':
				out.append(u'Delete "{}" from position {}'.format(ord(s[-1]),i))
			elif s[0]=='+':
				out.append(u'Add "{}" to position {}'.format(ord(s[-1]),i))
	with open(f"data_diff{idx}.txt", 'w') as file:
		file.write("\n".join(out))

In [6]:
np.unique((old_data[0]["turn_mask"]==old_data[0]["turn_mask"]))

array([ True])

In [5]:
len(new_data[0]["input_ids"])

512

In [6]:
new_data[0]["tokens"]

['[CLS]',
 '{entity_2}',
 ':',
 'hey',
 '!',
 'speaker',
 '2',
 ':',
 'hey',
 '.',
 'speaker',
 '3',
 ':',
 'hey',
 ',',
 'man',
 '.',
 'what',
 "'",
 's',
 'up',
 '?',
 '{entity_2}',
 ':',
 'maybe',
 'you',
 'can',
 'tell',
 'me',
 '.',
 'my',
 'agent',
 'would',
 'like',
 'to',
 'know',
 'why',
 'i',
 'didn',
 "'",
 't',
 'show',
 'up',
 'at',
 'the',
 'audition',
 'i',
 'didn',
 "'",
 't',
 'know',
 'i',
 'had',
 'today',
 '.',
 'the',
 'first',
 'good',
 'thing',
 'she',
 'gets',
 'me',
 'in',
 'weeks',
 '.',
 'how',
 'could',
 'you',
 'not',
 'give',
 'me',
 'the',
 'message',
 '?',
 '!',
 'speaker',
 '3',
 ':',
 'well',
 ',',
 'i',
 "'",
 'll',
 'tell',
 'ya',
 'i',
 'do',
 'enjoy',
 'guilt',
 ',',
 'but',
 ',',
 'ah',
 ',',
 'it',
 'wasn',
 "'",
 't',
 'me',
 '.',
 'speaker',
 '2',
 ':',
 'yes',
 ',',
 'it',
 'was',
 '!',
 'it',
 'was',
 'him',
 '!',
 'uh',
 'huh',
 '!',
 'okay',
 ',',
 'it',
 'was',
 'me',
 '!',
 '{entity_2}',
 ':',
 'how',
 'is',
 'it',
 'you',
 '?',
 'speaker

In [10]:
tokenizer.tokenize("joey tribbani")

['joey', 'tri', '##bba', '##ni']

In [9]:
tokenizer.convert_ids_to_tokens(old_data[0]["input_ids"])

['[CLS]',
 '[unused11]',
 ':',
 'hey',
 '!',
 'speaker',
 '2',
 ':',
 'hey',
 '.',
 'speaker',
 '3',
 ':',
 'hey',
 ',',
 'man',
 '.',
 'what',
 "'",
 's',
 'up',
 '?',
 '[unused11]',
 ':',
 'maybe',
 'you',
 'can',
 'tell',
 'me',
 '.',
 'my',
 'agent',
 'would',
 'like',
 'to',
 'know',
 'why',
 'i',
 'didn',
 "'",
 't',
 'show',
 'up',
 'at',
 'the',
 'audition',
 'i',
 'didn',
 "'",
 't',
 'know',
 'i',
 'had',
 'today',
 '.',
 'the',
 'first',
 'good',
 'thing',
 'she',
 'gets',
 'me',
 'in',
 'weeks',
 '.',
 'how',
 'could',
 'you',
 'not',
 'give',
 'me',
 'the',
 'message',
 '?',
 '!',
 'speaker',
 '3',
 ':',
 'well',
 ',',
 'i',
 "'",
 'll',
 'tell',
 'ya',
 'i',
 'do',
 'enjoy',
 'guilt',
 ',',
 'but',
 ',',
 'ah',
 ',',
 'it',
 'wasn',
 "'",
 't',
 'me',
 '.',
 'speaker',
 '2',
 ':',
 'yes',
 ',',
 'it',
 'was',
 '!',
 'it',
 'was',
 'him',
 '!',
 'uh',
 'huh',
 '!',
 'okay',
 ',',
 'it',
 'was',
 'me',
 '!',
 '[unused11]',
 ':',
 'how',
 'is',
 'it',
 'you',
 '?',
 'speaker

In [5]:
old_data[0].keys()

dict_keys(['input_ids', 'segment_ids', 'input_mask', 'speaker_ids', 'label_ids', 'mention_id', 'turn_mask', 'graph'])

In [6]:
new_data[0].keys()

dict_keys(['tokens', 'label_ids', 'input_ids', 'input_mask', 'segment_ids', 'speaker_ids', 'mention_ids', 'turn_masks', 'graph'])

In [47]:
cmp = 'turn_masks'
np.unique(old_data[0][cmp if not (cmp=="mention_ids" or cmp=="turn_masks") else ("mention_id" if cmp=="mention_ids" else "turn_mask")]==new_data[0][cmp], return_counts=True)


(array([False,  True]), array([  9716, 252428], dtype=int64))

In [92]:
old_data[0]['graph']

Graph(num_nodes={'node': 23},
      num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 24, ('node', 'speaker', 'node'): 114},
      metagraph=[('node', 'node', 'dialog'), ('node', 'node', 'entity'), ('node', 'node', 'speaker')])

In [ ]:
nx.draw(old_data[0]['graph'][('node', 'dialog', 'node')].to_networkx())

In [ ]:
new_graph = pickle.loads(new_data[0]['graph'])[0]
nx.draw(new_graph[('node', 'dialog', 'node')].to_networkx())

In [8]:
from difflib import ndiff
tete = "".join([chr(code) for code in old_data[0]['input_ids']])
g4g = "".join([chr(code) for code in new_data[0]['input_ids']])
for i,s in enumerate(ndiff(g4g,tete)):
	if s[0]==' ': continue
	elif s[0]=='-':
		print(u'Delete "{}" from position {}'.format(ord(s[-1]),i))
	elif s[0]=='+':
		print(u'Add "{}" to position {}'.format(ord(s[-1]),i))

In [6]:
print(tokenizer.convert_ids_to_tokens([10322, 25443]))
print(tokenizer.convert_ids_to_tokens([22861, 4775, 2072]))

['##bb', '##iani']
['bb', 'ian', '##i']


In [7]:
tokenizer.convert_ids_to_tokens([6216])
tokenizer.convert_ids_to_tokens([2222, 2063])

['ll', '##e']

In [109]:
new_data[0]['speaker_ids']

array([ 0, 12, 12, 12, 12,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 12, 12,
       12, 12, 12, 12, 12,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  2,  2,  2

In [110]:
old_data[0]['speaker_ids']

array([ 0, 12, 12, 12, 12,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2, 12, 12, 12, 12, 12, 12, 12,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4